In [ ]:
!pip install numpy==1.16.1
import numpy as np

     |████████████████████████████████| 17.3MB 9.1MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4


In [ ]:
from keras.datasets import imdb
from keras.models import Sequential 
from keras.layers import LSTM,Embedding,Dense,Dropout
from keras.preprocessing import sequence

Using TensorFlow backend.


## Load the dataset

In [ ]:
vocab_size=5000
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=vocab_size)

In [ ]:
x_train[0]

In [ ]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [ ]:
#dictionary
word2id=imdb.get_word_index()
id2word={i:word for word,i in word2id.items()}

1646592/1641221 [==============================] - 1s 0us/step


In [ ]:
review=[id2word.get(i,'') for i in x_train[15]]
print(review)

['the', 'is', 'rate', 'hope', 'br', 'my', 'out', 'double', 'this', 'is', 'again', 'overwhelming', 'country', 'known', 'portrayed', 'desperation', 'memories', 'is', 'and', 'and', 'anyway', 'standard', 'planet', 'poem', 'late', 'and', 'to', 'performance', 'not', 'his', "there's", 'is', 'carrey', 'and', 'just', 'being', 'film', 'jerry', 'in', 'jerry', 'baby', 'and', 'to', 'murders', 'this', 'is', 'and', 'to', 'have', 'into', 'one', 'as', 'fall', 'issues', 'but', 'is', 'on', 'was', 'had', "can't", 'is', 'dark', 'some', 'br', 'of', 'needed', 'based', 'just', 'table', 'and', 'and', 'of', 'little', 'with', 'barely', 'may', 'and', 'to', 'country', 'presentation', "don't", 'plot', 'and', 'he', '12', 'in', 'could', 'is', 'blow', 'opportunity', 'lie', 'wood', 'are', 'is', 'late', 'helps', 'ii', 'wanted', 'not', 'nothing', 'from', 'lost', 'and', "i'm", 'and', 'jenny', 'as', 'with', 'keep', 'but', 'and', 'and', 'that', 'effects', 'just', 'is', 'remember', 'b', 'was', 'off', 'use', 'of', 'and', 'mer

In [ ]:
#calculate max/min length of any review
mx=len(max(x_train+x_test,key=len))
mn=len(min(x_train+x_test,key=len))
#padding
maxlen=50
x_train1=sequence.pad_sequences(x_train, maxlen)
x_test1=sequence.pad_sequences(x_test,maxlen)

In [ ]:
#split dataset into train and validation set
valid_size=64
x_train2=x_train1[valid_size:]
x_val=x_train1[:valid_size]
y_train2=y_train[valid_size:]
y_valid=y_train[:valid_size]

## The model

In [ ]:
#model
embedding_size=32
model=Sequential()
model.add(Embedding(vocab_size,embedding_size,input_length=maxlen))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
print(model.summary())
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 32)            160000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

## Training

In [ ]:
num_epochs=1
batch_size=32
model.fit(x_train2,y_train2,validation_data=(x_val,y_valid),batch_size=batch_size,epochs=num_epochs,callbacks=[cp_callback])

Train on 24936 samples, validate on 64 samples
Epoch 1/1
24936/24936 [==============================] - 57s 2ms/step - loss: 0.1600 - acc: 0.9372 - val_loss: 0.5512 - val_acc: 0.7969

Epoch 00001: saving model to training_1/cp.ckpt


In [ ]:
scores=model.evaluate(x_test1,y_test,verbose=1)
print("Acurracy= "+str(scores[1]))

25000/25000 [==============================] - 24s 947us/step
Acurracy= 0.8056


## Testing

In [ ]:
x_runtime=np.array(['it','was','a','very','bad','movie','the','story','was','awful','and','the' ,'acting','was' ,'pathetic'],dtype='object')

In [ ]:
print(x_runtime)

['it' 'was' 'a' 'very' 'bad' 'movie' 'the' 'story' 'was' 'awful' 'and'
 'the' 'acting' 'was' 'pathetic']


In [ ]:
x_runtime=np.array([word2id.get(word,0) for word in x_runtime],dtype='object')
print(x_runtime)

[9 13 3 52 75 17 1 62 13 370 2 1 113 13 1232]


In [ ]:
x_runtime=x_runtime.reshape(1,x_runtime.shape[0])
x_runtime=sequence.pad_sequences(x_runtime,maxlen)


In [ ]:
print(model.predict(x_runtime))

[[0.8531893]]


In [ ]:
print(x_train.dtype)

object
